In [3]:
from Bio.PDB.ResidueDepth import ResidueDepth
from Bio.PDB.PDBParser import PDBParser
from pylab import *
from Bio import SeqIO
import matplotlib.pyplot as plt
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP
import pandas as pd
import os
import re
import gzip
import shutil
import glob
import numpy as np
import scipy.stats as stats
import pylab as pl

In [4]:
df_dssp= pd.read_csv('df_dssp_new.csv')


In [5]:
test_df= df_dssp.iloc[5:10]

In [6]:
test_df 

,Unnamed: 0,Sequence,ProteinID,Modifications,start,end,cleavage_sites,Last_AA,LastAA_position,Cleavage,...,T,2 Oxidation (M),3 Oxidation (M),Acetyl (Protein N-term),"Acetyl (Protein N-term),2 Oxidation (M)","Acetyl (Protein N-term),Oxidation (M)",Oxidation (M),Unmodified,res_depth,ca_depth
5,5,AAAAAAGAASGLPGPVAQGLK,Q96P70,Acetyl (Protein N-term),2,22,AAAAAAGAASGLPGPVAQGLK,K,22,0,...,0,0,0,1,0,0,0,0,2.249620,2.935947
6,6,AAAAAAVGPGAGGAGSAVPGGAGPCATVSVFPGAR,Q86X55,Acetyl (Protein N-term),2,36,AAAAAAVGPGAGGAGSAVPGGAGPCATVSVFPGAR,R,36,0,...,0,0,0,1,0,0,0,0,2.340984,3.085196
7,7,AAAAAEQQQFYLLLGNLLSPDNVVR,O00410,Acetyl (Protein N-term),2,26,AAAAAEQQQFYLLLGNLLSPDNVVR,R,26,0,...,0,0,0,1,0,0,0,0,2.520130,3.742286
8,8,AAAAAGTATSQR,Q9Y2Z0,Acetyl (Protein N-term),2,13,AAAAAGTATSQR,R,13,0,...,0,0,0,1,0,0,0,0,1.827747,1.999773
9,9,AAAAASASQDELNQLER,Q5VYK3,Acetyl (Protein N-term),2,18,AAAAASASQDELNQLER,R,18,0,...,0,0,0,1,0,0,0,0,1.873818,1.999583


In [38]:
def residue_depth(file): #This function calculates the residue depth of a given PDB file and makes a dataframe with the results
    parser = PDBParser() #Creates a parser object
    structure = parser.get_structure(f'{file}', file)   #Creates a structure object from the PDB file
    model = structure[0] #Creates a model object from the structure object
    rd = ResidueDepth(model)    #Creates a ResidueDepth object from the model object

    appender = []   #Creates a list to append the results to
    for k in rd.property_keys:  #Iterates through the keys in the ResidueDepth object
        x = rd.property_dict[k] #Creates a variable to store the value of the key
        resdepth = x[0] #Creates a variable to store the value of the first element of the list
        cadepth = x[1]  
        appender.append((resdepth, cadepth))    #Appends the results to the list

    df_rd = pd.DataFrame.from_records(appender, columns=['res_depth', 'ca_depth'])  #Creates a dataframe from the list of results
    df_rd['index'] = range(1, len(df_rd) + 1)   #Adds an index column to the dataframe
    return df_rd

In [46]:
def get_protein_name(df): #Function uses proteins in data frame to find all the PDB files in the directory and run the residue depth function on them
    proteinID= [] #List to store the protein IDs
    for p in df_dssp['ProteinID']:  #Loop through the proteins in the data frame
        proteinID.append(p) #Append the protein ID to the list

    proteins= set(proteinID)   #Convert the list to a set to remove duplicates
    for k in proteins:  #Loop through the set of proteins
        print(k)
        filename= f'AF-{k}-F1-model_v2.pdb' #Create the filename
        if filename in os.listdir('human_proteome/'):   #Check if the file is in the directory
            print(filename)
            df_rd= residue_depth(f'human_proteome/{filename}')  #Run the residue depth function on the file
            df_rd.to_csv(f'rd/{filename}.csv')  #Save the dataframe to a csv file

In [ ]:
def add_values(df): #Function to add the values of the residue depth and cadepth to the dataframe

    res_depth = [] #List to store the residue depth values
    ca_depth = []   
    for i,j in df_dssp.iterrows():  #Loop through the dataframe
        file= f'AF-{j["ProteinID"]}-F1-model_v2.pdb.csv'    #Create the filename

        if file in os.listdir('rd/'):   #Check if the file is in the directory

            df_rd= pd.read_csv(f'rd/{file}')    #Read the csv file
            if j['LastAA_position'] in df_rd['index']:  #Check if the last amino acid position is in the dataframe
                res_depth.append(df_rd.loc[df_rd['index'] == j['LastAA_position']]['res_depth'].values[0])  #Append the residue depth value to the list
                ca_depth.append(df_rd.loc[df_rd['index'] == j['LastAA_position']]['ca_depth'].values[0])    #Append the cadepth value to the list
            else:
                res_depth.append('') #Append a blank value to the list
                ca_depth.append('')  
        else:
            res_depth.append('')    #Append a blank value to the list
            ca_depth.append('') 

    print(res_depth)

    df_dssp['res_depth'] = res_depth    #Add the values to the dataframe
    df_dssp['ca_depth'] = ca_depth   #Add the values to the dataframe

In [ ]:
res_depth = []  #List to store the residue depth values
ca_depth = []
for i,j in df_dssp.iterrows(): #Loop through the data frame
    file= f'AF-{j["ProteinID"]}-F1-model_v2.pdb.csv'    #Create the filename

    if file in os.listdir('rd/'):   

        df_rd= pd.read_csv(f'rd/{file}')
        if j['LastAA_position'] in df_rd['index']:
            res_depth.append(df_rd.loc[df_rd['index'] == j['LastAA_position']]['res_depth'].values[0])
            ca_depth.append(df_rd.loc[df_rd['index'] == j['LastAA_position']]['ca_depth'].values[0])
        else:
            res_depth.append('')
            ca_depth.append('')
    else:
        res_depth.append('')
        ca_depth.append('')

print(res_depth)

df_dssp['res_depth'] = res_depth
df_dssp['ca_depth'] = ca_depth


In [12]:
sasa_value= [] #List to store the sasa values
for i,j in test_df.iterrows():  #Loop through the dataframe
    file= f'AF-{j["ProteinID"]}-F1-model_v2.pdb.csv'    #Create the filename

    if file in os.listdir('sasa/'):   #Check if the file is in the directory
        aa_df= pd.read_csv(f'sasa/{file}')  #Read the csv file
        if j['LastAA_position'] in aa_df['index']:  #Check if the last amino acid position is in the dataframe
            sasa_value.append(aa_df.loc[aa_df['index'] == j['LastAA_position']]['SASA'].values[0]) #Append the sasa value to the list
        else:
            sasa_value.append('') #Append a blank value to the list
    else:
        sasa_value.append('')

test_df['SASA']= sasa_value #Add the sasa values to the dataframe



<ipython-input-12-b6069b8671d3>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['SASA']= sasa_value #Add the sasa values to the dataframe


In [13]:
test_df

,Unnamed: 0,Sequence,ProteinID,Modifications,start,end,cleavage_sites,Last_AA,LastAA_position,Cleavage,...,2 Oxidation (M),3 Oxidation (M),Acetyl (Protein N-term),"Acetyl (Protein N-term),2 Oxidation (M)","Acetyl (Protein N-term),Oxidation (M)",Oxidation (M),Unmodified,res_depth,ca_depth,SASA
5,5,AAAAAAGAASGLPGPVAQGLK,Q96P70,Acetyl (Protein N-term),2,22,AAAAAAGAASGLPGPVAQGLK,K,22,0,...,0,0,1,0,0,0,0,2.249620,2.935947,68.40
6,6,AAAAAAVGPGAGGAGSAVPGGAGPCATVSVFPGAR,Q86X55,Acetyl (Protein N-term),2,36,AAAAAAVGPGAGGAGSAVPGGAGPCATVSVFPGAR,R,36,0,...,0,0,1,0,0,0,0,2.340984,3.085196,116.54
7,7,AAAAAEQQQFYLLLGNLLSPDNVVR,O00410,Acetyl (Protein N-term),2,26,AAAAAEQQQFYLLLGNLLSPDNVVR,R,26,0,...,0,0,1,0,0,0,0,2.520130,3.742286,94.10
8,8,AAAAAGTATSQR,Q9Y2Z0,Acetyl (Protein N-term),2,13,AAAAAGTATSQR,R,13,0,...,0,0,1,0,0,0,0,1.827747,1.999773,163.67
9,9,AAAAASASQDELNQLER,Q5VYK3,Acetyl (Protein N-term),2,18,AAAAASASQDELNQLER,R,18,0,...,0,0,1,0,0,0,0,1.873818,1.999583,168.90


In [51]:
df_dssp.to_csv('df_dssp_new.csv')

In [ ]:
df_dssp.to_csv('df_dssp1.csv')

In [17]:
from Bio.PDB.SASA import ShrakeRupley

p = PDBParser(QUIET=True)
structure = p.get_structure("AF-Q86X55-F1-model_v2", "human_proteome/AF-Q86X55-F1-model_v2.pdb")
sr = ShrakeRupley()
sr.compute(structure[0], level="R")
my_list = []


for chain in structure[0]:
    for res in chain:
        my_list.append((res.get_resname(),round(res.sasa,2)))



In [18]:
print(my_list[35])

('ARG', 116.54)


In [ ]:
# res_depth= res_depth: average depth of all atoms in a residue
# ca_depth= ca_depth:  depth of the alpha carbon atom